<a href="https://colab.research.google.com/github/TheMihirNaik/google-search-console-api-using-python/blob/main/5_Organic_CTR_Curve_using_GSC_API_%26_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSC API Class

In [ ]:
# Import required packages
from oauth2client.client import OAuth2WebServerFlow
from googleapiclient.discovery import build
import httplib2
import pandas as pd

class GoogleSearchConsole:
    def __init__(self, client_id, client_secret, oauth_scope, redirect_uri):
        self.webmasters_service = self.authenticate(client_id, client_secret, oauth_scope, redirect_uri)

    def authenticate(self, client_id, client_secret, oauth_scope, redirect_uri):
      try:
          # Authenticate with Google Search Console API
          flow = OAuth2WebServerFlow(client_id, client_secret, oauth_scope, redirect_uri)
          authorize_url = flow.step1_get_authorize_url()
          print("Go to the following link in your browser: " + authorize_url)
          auth_code = input("Enter your Authorization Code here:")
          credentials = flow.step2_exchange(auth_code)
          http = httplib2.Http()
          creds = credentials.authorize(http)
          webmasters_service = build('searchconsole', 'v1', http=creds)

          return webmasters_service

      except Exception as e:
          # Handle authentication errors
          print("Error during authentication:", str(e))
          return None

    def get_site_list(self):
        # Fetch the site data
        site_list = self.webmasters_service.sites().list().execute()

        all_sites = []
        for each in site_list['siteEntry']:
            all_sites.append(each['siteUrl'])

        return all_sites

    def fetch_search_console_data(self, website_url, start_date, end_date, dimensions, dimensionFilterGroups):
        # Initialize an empty list to store the rows from the response
        all_responses = []

        # Initialize the start row to 0
        start_row = 0

        # Loop until all rows have been retrieved
        while True:
            # Build the request body for the API call
            request_body = {
                "startDate": start_date,
                "endDate": end_date,
                "dimensions": dimensions,
                "dimensionFilterGroups": dimensionFilterGroups,
                "rowLimit": 25000,
                "dataState": "final",
                'startRow': start_row
            }

            # Call the API with the request body
            response_data = self.webmasters_service.searchanalytics().query(siteUrl=website_url, body=request_body).execute()

            # Append the rows from the response to the all_responses list
            for row in response_data['rows']:
                # Create a temporary list to hold the values for the row
                temp = []
                # Extract the values for the keys (dimensions)
                for key in row['keys']:
                    temp.append(key)
                # Extract the values for clicks, impressions, CTR, and position
                temp.append(row['clicks'])
                temp.append(row['impressions'])
                temp.append(row['ctr'])
                temp.append(row['position'])
                # Append the row to the all_responses list
                all_responses.append(temp)

            # Update the start row to reflect the number of rows retrieved
            start_row += len(response_data['rows'])

            # Print a progress message
            print("fetched up to " + str(start_row) + " rows of data")

            # Check if the number of rows retrieved is less than the row limit
            if len(response_data['rows']) < 25000:
                break

        # Create a DataFrame from the all_responses list, with columns corresponding to the requested dimensions and metrics
        df = pd.DataFrame(all_responses, columns=dimensions + ['clicks', 'impressions', 'ctr', 'position'])

        # Return the DataFrame
        return df

    def url_inspection(self, page_url, website_url, languageCode):

      request_body = {"inspectionUrl":page_url,"siteUrl": website_url,"languageCode":languageCode}

      # I learned to make this query from Tobias Willmann - Here is the LinkedIn Post he shared. https://www.linkedin.com/feed/update/urn:li:activity:6893990001873670144/
      json_output = self.webmasters_service.urlInspection().index().inspect(body=request_body).execute()

      return json_output


# GSC Credentials & Building GSC Service

In [ ]:
CLIENT_ID = "1085687348720-cpabg6ttpsqjji0j6pgbkb8sbcdcp4nt.apps.googleusercontent.com"
CLIENT_SECRET = "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"
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

In [ ]:
# Create an instance of the GoogleSearchConsole class
GSC_Service = GoogleSearchConsole(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)

Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=1085687348720-cpabg6ttpsqjji0j6pgbkb8sbcdcp4nt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&access_type=offline&response_type=code
Enter your Authorization Code here:4/1AfJohXlQgrXkcPBD4abJA21bzbAu-VJsomdEkVaUtF8t_UtR6X8yljFwXc4


# List of Properties in GSC

In [ ]:
# Get a list of sites
sites = GSC_Service.get_site_list()
sites

['https://www.monocubed.com/',
 'https://moneymonkdigital.com/',
 'https://www.tridentpbi.in/',
 'https://growthnatives.com/']

# Fetching GSC Data & Preparing DataFrame for Plotting CTR Curve

In [ ]:
# fetching site wide data
website_url = 'https://www.tridentpbi.in/'
start_date = '2022-01-01'
end_date = '2023-09-15'

dimensions = ['query']

dimensionFilterGroups = [{ "filters" : []}]


# Fetch and print search console data
df = GSC_Service.fetch_search_console_data(website_url, start_date, end_date, dimensions, dimensionFilterGroups)

fetched up to 6944 rows of data


In [ ]:
df

,query,clicks,impressions,ctr,position
0,trident paper box industries,1006,2141,0.469874,1.163008
1,types of corrugated boxes,338,4560,0.074123,6.769518
2,corrugated box price calculator,322,930,0.346237,3.436559
3,corrugated box costing formula pdf,275,1605,0.171340,3.871028
4,trident packaging,228,1468,0.155313,5.700954
...,...,...,...,...,...
6939,बॉक्स कट,0,4,0.000000,2.250000
6940,बॉक्स कटिंग,0,1,0.000000,1.000000
6941,हॉट कॉफी,0,1,0.000000,40.000000
6942,બોક્સ,0,1,0.000000,1.000000


In [ ]:
# round the position number
df['round_position'] = round(df['position'], 0)
df

,query,clicks,impressions,ctr,position,round_position
0,trident paper box industries,1006,2141,0.469874,1.163008,1.0
1,types of corrugated boxes,338,4560,0.074123,6.769518,7.0
2,corrugated box price calculator,322,930,0.346237,3.436559,3.0
3,corrugated box costing formula pdf,275,1605,0.171340,3.871028,4.0
4,trident packaging,228,1468,0.155313,5.700954,6.0
...,...,...,...,...,...,...
6939,बॉक्स कट,0,4,0.000000,2.250000,2.0
6940,बॉक्स कटिंग,0,1,0.000000,1.000000,1.0
6941,हॉट कॉफी,0,1,0.000000,40.000000,40.0
6942,બોક્સ,0,1,0.000000,1.000000,1.0


In [ ]:
# grouping the data by position
organic_ctr_df = df.groupby(['round_position']).agg(
    clicks = ('clicks', 'sum'),
    impressions = ('impressions', 'sum')
    ).reset_index()

#printing df
organic_ctr_df

,round_position,clicks,impressions
0,1.0,1086,3982
1,2.0,116,4893
2,3.0,702,7937
3,4.0,1010,14003
4,5.0,514,7845
...,...,...,...
116,156.0,0,1
117,160.0,0,1
118,170.0,0,1
119,182.0,0,17


In [ ]:
# Calculate CTR for each Position
organic_ctr_df['ctr'] = round((organic_ctr_df['clicks'] / organic_ctr_df['impressions'] * 100), 2)
organic_ctr_df

,round_position,clicks,impressions,ctr
0,1.0,1086,3982,27.27
1,2.0,116,4893,2.37
2,3.0,702,7937,8.84
3,4.0,1010,14003,7.21
4,5.0,514,7845,6.55
...,...,...,...,...
116,156.0,0,1,0.00
117,160.0,0,1,0.00
118,170.0,0,1,0.00
119,182.0,0,17,0.00


In [ ]:
#create a new Dataframe for Top 10 Positions
organic_ctr_df_top_10 = organic_ctr_df[organic_ctr_df['round_position'] < 11]
organic_ctr_df_top_10

,round_position,clicks,impressions,ctr
0,1.0,1086,3982,27.27
1,2.0,116,4893,2.37
2,3.0,702,7937,8.84
3,4.0,1010,14003,7.21
4,5.0,514,7845,6.55
5,6.0,654,28059,2.33
6,7.0,521,17753,2.93
7,8.0,236,18230,1.29
8,9.0,118,13206,0.89
9,10.0,67,9892,0.68


In [ ]:
import plotly.express as px
import pandas as pd

# Assuming you have the top_10_ctr_df dataframe

fig = px.bar(organic_ctr_df_top_10,
             y='round_position',
             x='ctr',
             orientation='h',
             title='Organic CTR %')

# Reverse the y-axis
fig.update_layout(yaxis=dict(autorange='reversed'))

# Set custom axis titles
fig.update_layout(xaxis=dict(title="Click Through Rate"), yaxis=dict(title="Ranking Position"))

# Add value labels to each bar
fig.update_traces(text=organic_ctr_df_top_10["ctr"])

fig.show()

# Preparing DF for Branded vs Non-branded Queries

In [ ]:
df

,query,clicks,impressions,ctr,position,round_position
0,trident paper box industries,1006,2141,0.469874,1.163008,1.0
1,types of corrugated boxes,338,4560,0.074123,6.769518,7.0
2,corrugated box price calculator,322,930,0.346237,3.436559,3.0
3,corrugated box costing formula pdf,275,1605,0.171340,3.871028,4.0
4,trident packaging,228,1468,0.155313,5.700954,6.0
...,...,...,...,...,...,...
6939,बॉक्स कट,0,4,0.000000,2.250000,2.0
6940,बॉक्स कटिंग,0,1,0.000000,1.000000,1.0
6941,हॉट कॉफी,0,1,0.000000,40.000000,40.0
6942,બોક્સ,0,1,0.000000,1.000000,1.0


In [ ]:
# function that determines whether the query is branded or non-branded
def keyword_type(query):
  # list of terms that defines branded queries
  brand_terms = ['trident', 'tpbi']

  # identifying if the query is a branded query
  for term in brand_terms:
    if term in query:
      return 'brand'

  #if it's not a branded query, then it's a non_braned query
  return 'non_brand'

In [ ]:
# apply this function to the dataframe & create a new column - query_type (branded or non-branded)
df['keyword_type'] = df['query'].apply(keyword_type)

# print df
df

,query,clicks,impressions,ctr,position,round_position,keyword_type
0,trident paper box industries,1006,2141,0.469874,1.163008,1.0,brand
1,types of corrugated boxes,338,4560,0.074123,6.769518,7.0,non_brand
2,corrugated box price calculator,322,930,0.346237,3.436559,3.0,non_brand
3,corrugated box costing formula pdf,275,1605,0.171340,3.871028,4.0,non_brand
4,trident packaging,228,1468,0.155313,5.700954,6.0,brand
...,...,...,...,...,...,...,...
6939,बॉक्स कट,0,4,0.000000,2.250000,2.0,non_brand
6940,बॉक्स कटिंग,0,1,0.000000,1.000000,1.0,non_brand
6941,हॉट कॉफी,0,1,0.000000,40.000000,40.0,non_brand
6942,બોક્સ,0,1,0.000000,1.000000,1.0,non_brand


In [ ]:
# prepare a dataframe to plot a chart - groupby
br_nb_organic_ctr_df = df.groupby(['round_position', 'keyword_type']).agg(
    clicks = ('clicks', 'sum'),
    impressions = ('impressions', 'sum')
).reset_index()

#print dataframe
br_nb_organic_ctr_df

,round_position,keyword_type,clicks,impressions
0,1.0,brand,1007,2149
1,1.0,non_brand,79,1833
2,2.0,brand,1,47
3,2.0,non_brand,115,4846
4,3.0,brand,47,242
...,...,...,...,...
159,156.0,non_brand,0,1
160,160.0,non_brand,0,1
161,170.0,non_brand,0,1
162,182.0,non_brand,0,17


In [ ]:
# Calculate CTR
br_nb_organic_ctr_df['ctr'] = round((br_nb_organic_ctr_df['clicks'] / br_nb_organic_ctr_df['impressions'] * 100),2)
br_nb_organic_ctr_df

,round_position,keyword_type,clicks,impressions,ctr
0,1.0,brand,1007,2149,46.86
1,1.0,non_brand,79,1833,4.31
2,2.0,brand,1,47,2.13
3,2.0,non_brand,115,4846,2.37
4,3.0,brand,47,242,19.42
...,...,...,...,...,...
159,156.0,non_brand,0,1,0.00
160,160.0,non_brand,0,1,0.00
161,170.0,non_brand,0,1,0.00
162,182.0,non_brand,0,17,0.00


In [ ]:
# Top 10 Position Dataframe
top_10_br_nb_organic_ctr_df = br_nb_organic_ctr_df[br_nb_organic_ctr_df['round_position'] < 11]
top_10_br_nb_organic_ctr_df

,round_position,keyword_type,clicks,impressions,ctr
0,1.0,brand,1007,2149,46.86
1,1.0,non_brand,79,1833,4.31
2,2.0,brand,1,47,2.13
3,2.0,non_brand,115,4846,2.37
4,3.0,brand,47,242,19.42
5,3.0,non_brand,655,7695,8.51
6,4.0,brand,0,56,0.00
7,4.0,non_brand,1010,13947,7.24
8,5.0,brand,29,573,5.06
9,5.0,non_brand,485,7272,6.67


In [ ]:
import plotly.express as px

fig = px.bar(top_10_br_nb_organic_ctr_df,
             x="ctr", y="round_position",
             title="Organic CTR Curve for Branded vs Non-branded Terms",
             color='keyword_type',
             barmode='group',  # This parameter groups the bars
             orientation='h',
             hover_data='ctr',
             text='ctr')

# Reverse the y-axis
fig.update_layout(yaxis=dict(autorange='reversed'))

# Set custom axis titles
fig.update_layout(xaxis=dict(title="Click Through Rate"), yaxis=dict(title="Ranking Position"))


fig.show()
